# Version Notifications:

date : 2020/11/19

key modifications :
1. Half the hidden size.

## packages, functions, utils 

In [1]:
import numpy as np
import pandas as pd 
import random
import os
import gc

from tqdm.notebook import tqdm
from scipy.special import erfinv

from sklearn.metrics import log_loss

import seaborn as sns
from matplotlib import pyplot as plt

import torch
from torch import nn
from torch.nn import functional as F
from torch.nn import Module
from torch import optim
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device : {device}')

import warnings 
warnings.filterwarnings('ignore')

Device : cuda


In [2]:
def cate2dummy(train, test, cate_features):
    df_cate = train[['sig_id'] + cate_features].append(test[['sig_id'] + cate_features])
    df_cate[cate_features] = df_cate[cate_features].astype(str)
    df_dumm = pd.get_dummies(df_cate, columns=cate_features)

    dumm_features = [c for c in df_dumm.columns if c not in ['sig_id']]
    train = train.merge(df_dumm, on='sig_id', how='left')
    test = test.merge(df_dumm, on='sig_id', how='left')
    
    return train, test, dumm_features

In [3]:
class GaussRankScaler():
    def __init__(self):
        self.epsilon = 0.001
        self.lower = -1 + self.epsilon
        self.upper = 1 - self.epsilon
        self.range = self.upper - self.lower
        
    def fit_transform(self, X):
        i = np.argsort(X, axis=0)
        j = np.argsort(i, axis=0)
        
        assert (j.min() == 0)
        assert (j.max() == len(j) - 1)
        
        j_range = len(j) - 1
        self.divider = j_range / self.range
        
        transformed = j / self.divider
        transformed = transformed - self.upper
        transformed = erfinv(transformed)
        
        return transformed
    
def gauss_rank_features(train, test, features):
    n_train = len(train)
    
    for f in tqdm(features):
        feat = train[f].append(test[f])
        feat_transformed = GaussRankScaler().fit_transform(feat)
        train[f] = feat_transformed.iloc[:n_train]
        test[f] = feat_transformed.iloc[n_train:]
        
    return train, test

In [4]:
from sklearn.model_selection._split import _BaseKFold
from sklearn.utils.validation import check_array
from sklearn.utils.multiclass import type_of_target
from sklearn.utils import check_random_state

def IterativeStratification(labels, r, random_state):
    """This function implements the Iterative Stratification algorithm described
    in the following paper:
    Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
    Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
    (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
    2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
    Heidelberg.
    """

    n_samples = labels.shape[0]
    test_folds = np.zeros(n_samples, dtype=int)

    # Calculate the desired number of examples at each subset
    c_folds = r * n_samples

    # Calculate the desired number of examples of each label at each subset
    c_folds_labels = np.outer(r, labels.sum(axis=0))

    labels_not_processed_mask = np.ones(n_samples, dtype=bool)

    while np.any(labels_not_processed_mask):
        # Find the label with the fewest (but at least one) remaining examples,
        # breaking ties randomly
        num_labels = labels[labels_not_processed_mask].sum(axis=0)

        # Handle case where only all-zero labels are left by distributing
        # across all folds as evenly as possible (not in original algorithm but
        # mentioned in the text). (By handling this case separately, some
        # code redundancy is introduced; however, this approach allows for
        # decreased execution time when there are a relatively large number
        # of all-zero labels.)
        if num_labels.sum() == 0:
            sample_idxs = np.where(labels_not_processed_mask)[0]

            for sample_idx in sample_idxs:
                fold_idx = np.where(c_folds == c_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                c_folds[fold_idx] -= 1

            break

        label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
        if label_idx.shape[0] > 1:
            label_idx = label_idx[random_state.choice(label_idx.shape[0])]

        sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]

        for sample_idx in sample_idxs:
            # Find the subset(s) with the largest number of desired examples
            # for this label, breaking ties by considering the largest number
            # of desired examples, breaking further ties randomly
            label_folds = c_folds_labels[:, label_idx]
            fold_idx = np.where(label_folds == label_folds.max())[0]

            if fold_idx.shape[0] > 1:
                temp_fold_idx = np.where(c_folds[fold_idx] ==
                                         c_folds[fold_idx].max())[0]
                fold_idx = fold_idx[temp_fold_idx]

                if temp_fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(temp_fold_idx.shape[0])]

            test_folds[sample_idx] = fold_idx
            labels_not_processed_mask[sample_idx] = False

            # Update desired number of examples
            c_folds_labels[fold_idx, labels[sample_idx]] -= 1
            c_folds[fold_idx] -= 1

    return test_folds


class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits, shuffle, random_state)

    def _make_test_folds(self, X, y):
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

        num_samples = y.shape[0]

        rng = check_random_state(self.random_state)
        indices = np.arange(num_samples)

        if self.shuffle:
            rng.shuffle(indices)
            y = y[indices]

        r = np.asarray([1 / self.n_splits] * self.n_splits)

        test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

        return test_folds[np.argsort(indices)]

    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)



In [5]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True

## Data Preparation

In [6]:
ON_KAGGLE = True

if ON_KAGGLE:
    input_dir = '/kaggle/input/lish-moa'
    output_dir = 'nn_04'
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
else:
    input_dir = '../_data/lish-moa'
    output_dir = 'output/nn_04'
    
    if not os.path.isdir('output'):
        os.mkdir('output')
        
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

In [7]:
os.listdir(input_dir)

['sample_submission.csv',
 'train_drug.csv',
 'train_targets_scored.csv',
 'train_targets_nonscored.csv',
 'train_features.csv',
 'test_features.csv']

In [8]:
train = pd.read_csv(os.path.join(input_dir, 'train_features.csv'))
test = pd.read_csv(os.path.join(input_dir, 'test_features.csv'))

targets_scored = pd.read_csv(os.path.join(input_dir, 'train_targets_scored.csv'))
targets_nonscored = pd.read_csv(os.path.join(input_dir, 'train_targets_nonscored.csv'))

submission = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'))

In [9]:
gene_features = [c for c in train.columns if c.startswith('g-')]
cell_features = [c for c in train.columns if c.startswith('c-')]
cate_features = ['cp_type', 'cp_time', 'cp_dose']
targets = [c for c in targets_scored if c not in ['sig_id']]

In [10]:
train = pd.merge(train, targets_scored, on='sig_id', how='left')

In [11]:
c = 'cp_type'
train.groupby(c)[targets].sum().sum(axis=1)

cp_type
ctl_vehicle        0
trt_cp         16844
dtype: int64

In [12]:
train = train.loc[train['cp_type'] != 'ctl_vehicle']
test = test.loc[test['cp_type'] != 'ctl_vehicle']

cate_features.remove('cp_type')

In [13]:
cp_time_map = {
    24: 0,
    48: 1,
    72: 2
}

train['cp_time'] = train['cp_time'].map(cp_time_map)
test['cp_time'] = test['cp_time'].map(cp_time_map)

In [14]:
train, test, dumm_features = cate2dummy(train, test, cate_features)

In [15]:
train, test = gauss_rank_features(train, test, gene_features+cell_features)

### Model Construction

In [16]:
# all_features = gene_features + cell_features + dumm_features + ['cp_time']
all_features = gene_features + cell_features

GRADIENT_ACCUMULATION_STEPS = 1
MAX_GRAD_NORM = 1000
HIDDEN_SIZE = 256
DROPOUT = 0.2
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 1e-6
BATCH_SIZE = 32
EPOCHS = 20

NUM_FEATURES = len(all_features)
NUM_TARGETS = len(targets)
NUM_FOLDS = 5

In [17]:
class MLP(Module):
    def __init__(self):
        super().__init__()
        self.lr1 = nn.utils.weight_norm(nn.Linear(NUM_FEATURES, HIDDEN_SIZE))
        self.bn1 = nn.BatchNorm1d(HIDDEN_SIZE)
        self.do1 = nn.Dropout(DROPOUT)
        self.pr1 = nn.PReLU()
        
        self.lr2 = nn.utils.weight_norm(nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE))
        self.bn2 = nn.BatchNorm1d(HIDDEN_SIZE)
        self.do2 = nn.Dropout(DROPOUT)
        self.pr2 = nn.PReLU()
        
#         self.lr3 = nn.utils.weight_norm(nn.Linear(HIDDEN_SIZE, NUM_TARGETS))
        self.lr3 = nn.Linear(HIDDEN_SIZE, NUM_TARGETS)
        
        
    def forward(self, x):
        x = self.lr1(x)
        x = self.bn1(x)
        x = self.do1(x)
        x = self.pr1(x)

        x = self.lr2(x)
        x = self.bn2(x)
        x = self.do2(x)
        x = self.pr2(x)
        
        x = self.lr3(x)
        
        return x

### Train and predict framework

In [18]:
class TrainDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data.values
        self.labels = labels.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = torch.FloatTensor(self.data[idx])
        y = torch.tensor(self.labels[idx]).float()
        
        return X, y

In [19]:
class TestDataset(Dataset):
    def __init__(self, data):
        self.data = data.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = torch.FloatTensor(self.data[idx])
        return X

In [20]:
class AverageMeter(object):
    """Compute and stores the average and current value"""
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = (self.sum / self.count)

In [21]:
def train_fn(train_loader,
             model,
             optimizer,
             epoch,
             scheduler,
             device):
    losses = AverageMeter()
    
    model.train()
    
    for step, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        
        batch_size = X.size(0)
        pred = model(X)
        loss = nn.BCEWithLogitsLoss()(pred, y)
        losses.update(loss.item(), batch_size)
        
        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss /= GRADIENT_ACCUMULATION_STEPS
        
        loss.backward()
        
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scheduler.step()
            optimizer.step()
            optimizer.zero_grad()
            
    return losses.avg

In [22]:
def validate_fn(valid_loader, model, device):
    losses = AverageMeter()
    model.eval()
    val_preds = []
    
    for step, (X, y) in enumerate(valid_loader):
        X, y = X.to(device), y.to(device)
        batch_size = X.size(0)
                
        with torch.no_grad():
            pred = model(X)
            
        loss = nn.BCEWithLogitsLoss()(pred, y)
        
        losses.update(loss.item(), batch_size)
        
#         tmp = pred.sigmoid().detach().cpu().numpy()
#         print(tmp.shape)
#         val_preds.append(tmp)
        val_preds.append(pred.sigmoid().detach().cpu().numpy())
        
        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss /= GRADIENT_ACCUMULATION_STEPS
            
    val_preds = np.concatenate(val_preds)
    
    return losses.avg, val_preds

In [23]:
def inference_fn(test_loader, model, device):
    model.eval()
    preds = []

    for step, (X) in enumerate(test_loader):
        X = X.to(device)
        
        with torch.no_grad():
            pred = model(X)
            
        preds.append(pred.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds)
    
    return preds

In [24]:
def run_single_fold(train,
                    test,
                    features,
                    targets,
                    device,
                    fold_num=0,
                    seed=42):
    
    model_file = os.path.join(output_dir, f'fold{fold_num}_seed{seed}.pth')
    seed_everything(seed)
    
    train_index = train.loc[train['fold'] != fold_num].index
    valid_index = train.loc[train['fold'] == fold_num].index
    
    train_data = train.loc[train_index][features].reset_index(drop=True)
    valid_data = train.loc[valid_index][features].reset_index(drop=True)
    
    train_targets = train.loc[train_index][targets]
    valid_targets = train.loc[valid_index][targets]
    
    train_dataset = TrainDataset(train_data, train_targets)
    valid_dataset = TrainDataset(valid_data, valid_targets)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, drop_last=False)
    
    # model
    model = MLP()
    model.to(device)
    
    optimizer = optim.Adam(model.parameters(),
                           lr=LEARNING_RATE,
                           weight_decay=WEIGHT_DECAY)
    
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                              pct_start=0.1,
                                              div_factor=1e3,
                                              max_lr=1e-2,
                                              epochs=EPOCHS,
                                              steps_per_epoch=len(train_loader))
    

    # train & valid
    best_loss = np.inf
    for epoch in range(EPOCHS):
        train_loss = train_fn(train_loader,
                              model,
                              optimizer,
                              epoch,
                              scheduler,
                              device)
        
        valid_loss, valid_pred = validate_fn(valid_loader, model, device)        
        
        if valid_loss < best_loss:
            print(f'epoch{epoch} save best model ... {train_loss}, {valid_loss}')
            best_loss = valid_loss
            oof = np.zeros((len(train), len(targets)))
            oof[valid_index] = valid_pred
            torch.save(model.state_dict(), model_file)
            
    # predictions
    test_dataset = TestDataset(test[features])
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    
    model = MLP()
    model.load_state_dict(torch.load(model_file))
    model.to(device)
    
    predictions = inference_fn(test_loader, model, device)
    
    # delete
    torch.cuda.empty_cache()
    
    return oof, predictions

In [25]:
def run_oof(train,
            test, 
            features,
            targets,
            device,
            n_fold=5,
            seed=42):
    
    oof = np.zeros((len(train), len(targets)))
    predictions = np.zeros((len(test), len(targets)))
        
    train['fold'] = 0
    mlskf = MultilabelStratifiedKFold(n_splits = NUM_FOLDS, shuffle=True, random_state=seed)
    for n, (train_index, valid_index) in enumerate(mlskf.split(train, train[targets])):
        train.loc[valid_index, 'fold'] = int(n)
        
    for _fold in range(n_fold):
        print(f'Fold {_fold+1}')
        _oof, _predictions = run_single_fold(train,
                                             test,
                                             features,
                                             targets,
                                             device,
                                             fold_num=_fold,
                                             seed=seed)
        oof += _oof
        predictions += (_predictions / n_fold)
        
    score = 0
    for i, t in enumerate(targets):
        _score = log_loss(train[t].values, oof[:, i])
        score += (_score / NUM_TARGETS)
    
    print(f'CV score: {score}')
    
    return oof, predictions

In [26]:
# Seed Averaging for solid result

oof = np.zeros((len(train), NUM_TARGETS))
predictions = np.zeros((len(test), NUM_TARGETS))

SEED = [0, 1, 2]

for seed in SEED:
    _oof, _predictions = run_oof(train,
                                 test,
                                 all_features,
                                 targets,
                                 device,
                                 n_fold=5,
                                 seed=seed)
    
    oof += (_oof / len(SEED))
    predictions += (_predictions / len(SEED))
    
score = 0
for i, t in enumerate(targets):
    _score = log_loss(train[t].values, oof[:, i])
    score += (_score / NUM_TARGETS)
    
print(f'Seed Averaged CV score : {score}')

Fold 1
epoch0 save best model ... 0.17508736338439215, 0.019126924133791037
epoch1 save best model ... 0.01920696488839921, 0.018345951684724093
epoch2 save best model ... 0.018470045422156252, 0.018054482946161746
epoch3 save best model ... 0.017699001275383642, 0.017430316765471618
epoch5 save best model ... 0.016607457698819085, 0.017112967884560588
epoch6 save best model ... 0.01627603340316156, 0.01685258479885236
Fold 2
epoch0 save best model ... 0.17527319428198257, 0.019047887119899364
epoch1 save best model ... 0.0191508216875177, 0.01867939341294722
epoch2 save best model ... 0.018442467765977782, 0.018165439084226572
epoch3 save best model ... 0.017585459894357915, 0.01790745940426945
epoch4 save best model ... 0.01701124234933977, 0.017423877034206465
epoch5 save best model ... 0.016597207647537554, 0.01712571407544583
epoch8 save best model ... 0.015714313032744575, 0.01711906118441087
epoch9 save best model ... 0.015490232254752385, 0.017094856923826736
epoch11 save best 

In [27]:
train[targets] = oof
train[['sig_id'] + targets].to_csv(os.path.join(output_dir, 'oof.csv'), index=False)

for c in targets:
    test[c] = 0
    
test[targets] = predictions
test[['sig_id'] + targets].to_csv(os.path.join(output_dir, 'pred.csv'), index=False)

In [28]:
result = targets_scored.drop(columns=targets).merge(train[['sig_id'] + targets], on='sig_id', how='left').fillna(0)

y_true = targets_scored[targets].values
y_pred = result[targets].values

score = 0
for i in range(NUM_TARGETS):
    _score = log_loss(y_true[:, i], y_pred[:, i])
    score += (_score / NUM_TARGETS)
    
print(f'Final result : {score}')

Final result : 0.014839685747266391


### Submit

In [29]:
if ON_KAGGLE:
    sub_file = 'submission.csv'
else:
    sub_file = os.path.join(output_dir, 'submission.csv')

In [30]:
sub = submission.drop(columns=targets).merge(test[['sig_id'] + targets], on='sig_id', how='left').fillna(0)
sub.to_csv(sub_file, index=False)
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000546,0.001132,0.003094,0.013157,0.015715,0.004966,0.001095,0.006447,0.000092,...,0.000777,0.001315,0.004643,0.000822,0.000381,0.000568,0.000549,0.001466,0.009638,0.001732
1,id_001897cda,0.000398,0.000658,0.001894,0.001871,0.001644,0.001835,0.004666,0.013775,0.011285,...,0.000551,0.000990,0.003552,0.000219,0.013174,0.000378,0.008563,0.000687,0.005651,0.004113
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000549,0.000378,0.002366,0.016651,0.016988,0.004359,0.002313,0.003993,0.000086,...,0.000670,0.001146,0.004432,0.029736,0.006751,0.000452,0.001074,0.002985,0.000822,0.004168
4,id_0027f1083,0.001195,0.001355,0.002352,0.015403,0.016986,0.004392,0.007876,0.002547,0.000402,...,0.000836,0.000436,0.005007,0.001417,0.000817,0.000712,0.001300,0.001711,0.000079,0.002067
